In [83]:
import pandas as pd
import numpy as np


with open('/gpfs/gpfs0/project/gpaa/machine_learning/may_2021_repo/feature_selection/train_test_splits_LAD_b13.csv', 'r') as f:
  train_test_splits = pd.read_csv(f)

with open('/gpfs/gpfs0/project/gpaa/machine_learning/may_2021_repo/preprocessing/AA_labeled_b13_fixed.csv', 'r') as f:
  df_old = pd.read_csv(f)

with open('/project/gpaa/machine_learning/jainam_capstone/preprocessing/LAD_labeled_batches0-16.csv', 'r') as f:
  df_new = pd.read_csv(f)

In [70]:
df_labeled = df_new.loc[~df_new['sample_id'].isin(df_old['sample_id'])]
df_labeled

,AGE,sample_id,ENSG00000186092,ENSG00000284733,ENSG00000284662,ENSG00000187634,ENSG00000187961,ENSG00000187583,ENSG00000188290,ENSG00000188157,...,ENSG00000160183,ENSG00000274175,ENSG00000277836,binary_pathology,female,male,asian,black,hispanic,white
33,45.0,5560-LM1,-0.238754,0.443763,1.898444,2.729031,0.353265,1.314989,-0.015695,1.527239,...,1.143280,0.395511,-0.505758,0,0,1,0,0,0,1
34,35.0,5570-LM1,-0.238754,-0.480220,-0.523113,-0.235011,-0.582148,-0.296967,-0.551963,-1.137759,...,-0.517262,-0.682812,-0.450100,0,0,1,0,0,0,1
35,36.0,5580-LM1,-0.238754,0.671380,-0.049661,-0.442233,-0.173853,1.263741,-0.195421,-0.205933,...,0.793437,-0.682812,0.057427,0,0,1,0,0,0,1
36,46.0,5590-LM1,0.236302,-0.480220,-0.523113,0.293941,0.892203,1.050530,1.942699,1.517264,...,-0.250158,0.905864,-0.147162,0,0,1,0,0,1,0
37,20.0,5600-LM1,-0.238754,-0.411525,-0.494870,-0.543378,-0.073526,-0.436827,0.663583,1.874316,...,-0.517262,-0.374372,-0.527529,0,0,1,0,1,0,0
38,39.0,5610-LM1,-0.238754,1.120741,-0.058782,-0.900066,-0.097270,-0.687943,-0.280856,-0.214434,...,-0.517262,-0.682812,-0.527529,0,0,1,0,0,0,1
39,32.0,5620-LM1,-0.238754,-0.480220,-0.523113,1.832199,0.194925,-0.338166,0.569005,1.540734,...,0.252948,3.494891,-0.449762,0,1,0,0,0,0,1
40,48.0,5630-LM1,-0.238754,-0.480220,-0.523113,-0.596670,-0.434899,-0.922136,0.690720,0.209445,...,-0.297056,-0.682812,0.000527,0,0,1,0,1,0,0
41,32.0,5640-LM1,-0.238754,-0.480220,-0.523113,-0.082552,-0.038383,0.135902,-0.437302,1.205326,...,2.018793,1.968592,2.453970,1,0,1,0,0,0,1
74,22.0,5650-LADLM1,-0.238754,2.173540,0.567914,-0.705904,-0.234090,-0.653674,0.840193,0.081700,...,-0.517262,1.428570,-0.403476,0,0,1,0,1,0,0


In [71]:
from sklearn.model_selection import StratifiedKFold
import random

random.seed(42)
skf = StratifiedKFold(n_splits = 5)

skf.get_n_splits(df_labeled, df_labeled['binary_pathology']) # get the same splits as the xgboost ones above
fold = 1
target = df_labeled['binary_pathology']

split = 0

split_dict = {}

for train_index, test_index in skf.split(df_labeled, target):
  split = split + 1
#for split in range(1,6):
  print("SPLIT " + str(split))

  train_col = [col for col in train_test_splits.columns if str(split)+'_train' in col]
  train_indices = train_test_splits[train_col[0]].dropna().to_list()
  test_col = [col for col in train_test_splits.columns if str(split)+'_test' in col]
  test_indices = train_test_splits[test_col[0]].dropna().to_list()
  
  X_train_old, X_test_old = df_old.iloc[train_indices], df_old.iloc[test_indices]

  train_indices = df_new.index[df_new['sample_id'].isin(X_train_old['sample_id'])].tolist()
  test_indices = df_new.index[df_new['sample_id'].isin(X_test_old['sample_id'])].tolist()
    
  train_set = set(train_indices)
  train_set |= set(train_index)
  
  test_set = set(test_indices)
  test_set |= set(test_index)
    
  split_dict["Split_"+str(split)+"_train"] = train_set
  split_dict["Split_"+str(split)+"_test"] = test_set
    
    
df_out = pd.DataFrame.from_dict(split_dict, orient='index')

df_out = df_out.transpose()

SPLIT 1
SPLIT 2
SPLIT 3
SPLIT 4
SPLIT 5


/home/jhm3ab/.conda/envs/gpaa/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [72]:
df_out

,Split_1_train,Split_1_test,Split_2_train,Split_2_test,Split_3_train,Split_3_test,Split_4_train,Split_4_test,Split_5_train,Split_5_test
0,0.0,64.0,0.0,65.0,0.0,0.0,0.0,5.0,0.0,32.0
1,1.0,0.0,1.0,5.0,1.0,1.0,1.0,8.0,1.0,19.0
2,2.0,66.0,2.0,6.0,2.0,2.0,2.0,9.0,2.0,20.0
3,3.0,67.0,3.0,71.0,3.0,3.0,3.0,10.0,3.0,21.0
4,4.0,1.0,4.0,72.0,4.0,4.0,4.0,11.0,4.0,22.0
...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,NaN,NaN,70.0,NaN,69.0,NaN,NaN,NaN
57,NaN,NaN,NaN,NaN,71.0,NaN,70.0,NaN,NaN,NaN
58,NaN,NaN,NaN,NaN,72.0,NaN,71.0,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN


In [73]:
df_out.to_csv("/project/gpaa/machine_learning/jainam_capstone/feature_selection/train_test_splits_LAD_b16_old.csv", index=False)

In [84]:
from sklearn.model_selection import StratifiedKFold
import random

random.seed(42)
skf = StratifiedKFold(n_splits = 5)

skf.get_n_splits(df_new, df_new['binary_pathology']) # get the same splits as the xgboost ones above
fold = 1
target = df_new['binary_pathology']

split = 0

split_dict = {}

for train_index, test_index in skf.split(df_new, target):
  split = split + 1
#for split in range(1,6):
  print("SPLIT " + str(split))
    
  split_dict["Split_"+str(split)+"_train"] = train_index
  split_dict["Split_"+str(split)+"_test"] = test_index
    
df_out = pd.DataFrame.from_dict(split_dict, orient='index')

df_out = df_out.transpose()

SPLIT 1
SPLIT 2
SPLIT 3
SPLIT 4
SPLIT 5


In [85]:
df_out

,Split_1_train,Split_1_test,Split_2_train,Split_2_test,Split_3_train,Split_3_test,Split_4_train,Split_4_test,Split_5_train,Split_5_test
0,18.0,0.0,0.0,18.0,0.0,35.0,0.0,51.0,0.0,67.0
1,19.0,1.0,1.0,19.0,1.0,36.0,1.0,52.0,1.0,70.0
2,20.0,2.0,2.0,20.0,2.0,37.0,2.0,56.0,2.0,72.0
3,21.0,3.0,3.0,21.0,3.0,38.0,3.0,57.0,3.0,75.0
4,22.0,4.0,4.0,22.0,4.0,39.0,4.0,58.0,4.0,76.0
...,...,...,...,...,...,...,...,...,...,...
67,85.0,NaN,85.0,NaN,85.0,NaN,85.0,NaN,68.0,NaN
68,86.0,NaN,86.0,NaN,86.0,NaN,86.0,NaN,69.0,NaN
69,87.0,NaN,87.0,NaN,87.0,NaN,87.0,NaN,71.0,NaN
70,88.0,NaN,88.0,NaN,88.0,NaN,88.0,NaN,73.0,NaN


In [86]:
df_out.to_csv("/project/gpaa/machine_learning/jainam_capstone/feature_selection/train_test_splits_LAD_b16_new.csv", index=False)